# Homework 4

**Name:** Omar Alejandro Guzmán Munguía

**e-mail:** omar.guzman5063@alumnos.udg.mx

# INSTRUCTIONS

**Minimum requirements:** 

- **Functions** that return the different types of trajectories: Brownian Motion (**BM**),
Correlated Random Walk (**CRW**), and Lévy Flight (**LF**).
- All functions must take as arguments the **number of steps**, **speed**, and **starting
position**.
    - In addition, **CRW** and **LF** functions must also take as an argument the **Cauchy
coefficient**.
    - Lastly, the **LF** function must also include the **Lévy exponent** (alpha) as an
argument.

- **Functions** that compute the metrics: Path length (**PL**), Mean Squared Displacement
(**MSD**), and Turning Angle Distribution (**TAD**).

**The dashboard should include at least the following functionalities:**
- A panel to display the current trajectory (3D projection).
- A panel to display the graphic for the metric of choice.
- Radio Items (or a similar type of selector) to select the type of trajectory (BM, CRW, LF).
- Drop down menu (or similar) to select the metric to be displayed (PL, MSD, TAD).
- Widgets to the trajectories’ parameters (integer and floating point values). These widgets
must be dynamically displayed according to the type of trajectory selected.

# MODULES

In [ ]:
import panel as pn
pn.extension()

import panel.widgets as pnw

import pandas as pd

import numpy as np

import plotly.graph_objects as go
pn.extension('plotly')

import math
from scipy.stats import exponweib
from scipy.stats import levy_stable
from scipy.spatial import distance

# CLASSES

In [18]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

In [27]:
###############################################################################################
# Brownian Motion Trajectory
###############################################################################################
def bm_2d(n_steps=1000, speed=6, s_x_pos=0, s_y_pos=0):
    """
    Arguments:
        n_steps: 
        speed: 
        s_pos: 
    Returns:
        BM_2d_df: 
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)

    BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)


    for i in range(n_steps-1):        
        turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
        
    # return dataframe
    return BM_2d_df

In [20]:
#####################################################################################
# Correlated Random Walk (CRW)
#####################################################################################
def crw_trajectory(n_steps=1000, speed=5, correlation_coeff=0.9, s_pos=0.0):
    """
    Arguments
    -----------
        n_steps: Number of steps in the trajectory.
        speed: Length of each step.
        correlation_coeff: Correlation coefficient (between -1 and 1) that controls the correlation between consecutive steps.
        s_pos: Initial direction of movement in radians (default is 0.0).

    Returns
    -----------
        crw_df: A pandas DataFrame with columns ['x', 'y'] representing the trajectory.
    """
    # Initialize the trajectory with the starting point at the origin
    trajectory = [Vec2d(0.0, 0.0)]
    
    # Initialize the current direction
    current_direction = s_pos
    
    for _ in range(n_steps):
        # Generate a random angle change based on the correlation coefficient
        angle_change = np.random.normal(0, np.sqrt(1 - correlation_coeff**2))
        
        # Update the current direction
        current_direction += angle_change
        
        # Calculate the next step vector
        next_step = Vec2d(speed, 0).rotated(current_direction)
        
        # Update the trajectory
        next_position = trajectory[-1] + next_step
        trajectory.append(next_position)
    
    # Convert the trajectory to a pandas DataFrame
    crw_df = pd.DataFrame([(pos.x, pos.y) for pos in trajectory], columns=['x', 'y'])
    
    
    return crw_df

In [21]:
#####################################################################################
# Path length
#####################################################################################
def path_length(trajectory_df, s_pos=[0]):
    """
    Arguments
    -----------
        trajectory_df: DataFrame with the x and y positions of the trajectory.
        s_pos: Starting position of the trajectory.

    Returns
    -----------
        path_length: Length of the trajectory.
    """
    # Initialize the path length
    path_length = pd.DataFrame(columns=['y_pos'])
    df = pd.DataFrame([{'y_pos':s_pos[0]}])

    path_length = pd.concat([path_length, df], ignore_index=True)

    for i in range(len(trajectory_df)-1):
        df = pd.DataFrame([{'y_pos':path_length.y_pos[i] + distance.euclidean(trajectory_df.iloc[i], trajectory_df.iloc[i+1])}])
        path_length = pd.concat([path_length, df], ignore_index=True)
        
    return path_length

In [22]:
#####################################################################################
# Mean Squared Displacement (MSD)
#####################################################################################
def msd(trajectory_df):
    """    
    Arguments
    -----------
        trajectory_df: DataFrame with columns 
    
    Returns
    -----------
        msd_df: DataFrame with columns ['tau', 'msd'] (time lag vs MSD).
    """
    x_col = 'x' if 'x' in trajectory_df.columns else 'x_pos'
    y_col = 'y' if 'y' in trajectory_df.columns else 'y_pos'
    
    # Extract coordinates
    x = trajectory_df[x_col].values
    y = trajectory_df[y_col].values
    n = len(x)
    
    msd_values = np.zeros(n - 1)  # Store MSD values
    taus = np.arange(1, n)  # All possible time lags
    
    for tau in taus:
        displacements = (x[tau:] - x[:-tau])**2 + (y[tau:] - y[:-tau])**2
        msd_values[tau - 1] = np.mean(displacements)  # Store MSD value for this tau
    
    return pd.DataFrame({'tau': taus, 'msd': msd_values})  

In [23]:
#####################################################################################
# Turning angles
#####################################################################################
def turning_angles(trajectory_df):
    """    
    Arguments
    -----------
        trajectory_df: DataFrame with 'x' and 'y' columns
    
    Returns
    -----------
        numpy: Array of turning angles
    """
    x = trajectory_df['x'].values
    y = trajectory_df['y'].values
    
    # Calculate directions (angles) between consecutive points
    dx = np.diff(x)
    dy = np.diff(y)
    directions = np.arctan2(dy, dx)
    
    # Calculate turning angles (differences between consecutive directions)
    turning_angles = np.diff(directions)
    
    # Normalize angles to be between -π and π
    turning_angles = np.mod(turning_angles + np.pi, 2 * np.pi) - np.pi
    
    return turning_angles

# WIDGETS

In [53]:
n_steps = pnw.IntSlider(name='Number of steps', width=160, value=500, step=100, start=10, end=1000)
speed = pnw.IntSlider(name="Speed", width=160, value=5, step=1, start=1, end=10)
s_x_pos = pnw.IntInput(name='xInitPos', width=70, value=0, step=1, start=-500, end=500)
s_y_pos = pnw.IntInput(name='yInitPos', width=70, value=0, step=1, start=-500, end=500)

metrics_input = pnw.Select(name='Metrics type', options=['PL', 'MSD', 'TAD'], value='PL', width=160)

@pn.depends(n_steps, speed, s_x_pos, s_y_pos)
def plot_traj(n_steps, speed, s_x_pos, s_y_pos):
    rw_df = bm_2d(n_steps, speed, s_x_pos, s_y_pos)

    fig_rw = go.Figure()
    fig_rw.add_trace(go.Scatter3d(x=rw_df.x_pos,
                                  y=rw_df.y_pos,
                                  z=rw_df.index,                                  
                                  mode='lines',
                                  line=dict(color='blue', width=2),
                                  ))
    return fig_rw

miniPanel = pn.Column(pn.Row(n_steps, speed, s_x_pos, s_y_pos), metrics_input, plot_traj)
miniPanel.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'2bf23738-4586-4558-a9e4-d1909ff8ee20': {'version…

In [ ]:
y_input = pnw.IntInput(name='YinitPos', width=70, value=0, step=1, start=-500, end=500)
s_x_pos = pnw.IntInput(name='Starting x position', value=0, step=1, start=-100, end=100)